In [1]:
import os

In [2]:
import fnmatch
import os
import os.path
import re

def walknroll(fun):
    includes = ['*.html', '*.md', '*.Rmd']
    excludes = ['*all_static*', '*_includes*', '*_layouts*', '*_drafts*']

    # transform glob patterns to regular expressions
    includes = r'|'.join([fnmatch.translate(x) for x in includes])
    excludes = r'|'.join([fnmatch.translate(x) for x in excludes]) or r'$.'

    for root, dirs, files in os.walk('..'):
        # exclude/include files
        files = [os.path.join(root, f) for f in files]
        files = [f for f in files if not re.match(excludes, f)]
        files = [f for f in files if re.match(includes, f)]

        for fname in files:
            fun(fname)

In [10]:
from collections import OrderedDict as OD

class NotPost(Exception):
    pass

class Post:
    def __init__(self, fn):
        self.fn = fn
        content = open(fn).read()
        if '---' not in content:
            raise NotPost
        self.fm = content.split('---')[1] # front matter
        self.body = content.split('---')[2]
        self.fm_vars = OD((l.split(':')[0], l.split(': ')[1]) for l in self.fm.split('\n') if ': ' in l)
    
    def head(self):
        return ('---\n' +
            '\n'.join([k+': '+v for k, v in self.fm_vars.iteritems()]) +
            '\n---')
    
    def _content(self):
        return self.head() + self.body

    def __str__(self):
        return self._content()
    
    def __repr__(self):
        return 'Post(\'{}\')'.format(self.fn)
    
    def save(self):
        with open(self.fn, 'w') as f:
            f.write(self._content())

In [13]:
def process_posts(fn):
    try:
        p = Post(fn)
    except NotPost:
        pass
    except Exception as e:
        print fn
        raise e
    else:
        if 'permalink' in p.fm_vars and 'javascript-graphing' in p.fm_vars['permalink']:
            p.fm_vars['redirect_from'] = p.fm_vars['permalink']
            p.fm_vars['permalink'] = p.fm_vars['permalink'].replace('javascript-graphing-library', 'javascript')
            p.save()

In [14]:
walknroll(process_posts)